# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [58]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
# Module for reading CSV files
import csv
csvpath = os.path.join("..","WeatherPy/output_data/cities.csv")
weather_df = pd.read_csv(csvpath)
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Max Temp,Lat,Lng,Wind Speed
0,Erbaa,40,TR,1579922363,69,33.80,40.67,36.57,6.93
1,Hofn,100,IS,1579922363,97,36.28,64.25,-15.21,44.52
2,East London,40,ZA,1579922303,77,69.80,-33.02,27.91,4.70
3,Mataura,72,NZ,1579922329,58,66.99,-46.19,168.86,10.00
4,Coromandel,100,BR,1579922364,98,66.63,-18.47,-47.20,9.69


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
# humidity heatmap
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df['Humidity'].astype(float)

# plot heatmap and set zoom level and coordinates
fig = gmaps.figure(center=(0,0),zoom_level=2.2)

# create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)
# add heat layer
fig.add_layer(heat_layer)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [147]:
# Create new DataFrame fitting weather criteria
# max temperature lower than 80 degrees but higher than 60.
# Wind speed less than 10 mph.
# Zero cloudiness.
ideal_df = weather_df.copy()
ideal_df = ideal_df[ideal_df['Max Temp'] >= 60] #& ideal_df[ideal_df['Max Temp'] <= 80]
ideal_df = ideal_df[ideal_df['Max Temp'] <= 80]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] == 0]
ideal_df = ideal_df.dropna()
ideal_df = ideal_df.reset_index()
ideal_df


,index,City,Cloudiness,Country,Date,Humidity,Max Temp,Lat,Lng,Wind Speed
0,11,Hermanus,0,ZA,1579922195,96,64.00,-34.42,19.23,1.01
1,20,Bonthe,0,SL,1579922367,89,75.78,7.53,-12.51,2.77
2,33,Taltal,0,CL,1579922370,79,69.78,-25.40,-70.48,2.10
3,49,Cidreira,0,BR,1579922374,70,72.99,-30.18,-50.21,2.13
4,81,Rocha,0,UY,1579922381,49,70.83,-34.48,-54.33,7.90
5,83,Port Alfred,0,ZA,1579922351,94,64.00,-33.59,26.89,2.30
6,98,Pozo Colorado,0,PY,1579922385,50,75.99,-23.49,-58.80,4.72
7,128,Upata,0,VE,1579922392,79,70.50,8.01,-62.40,3.91
8,190,Nicoya,0,CR,1579922407,73,77.00,10.15,-85.45,3.36
9,229,Port Lincoln,0,AU,1579922180,48,75.43,-34.73,135.87,3.85


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [148]:
# reset index and create "hotel_df"
hotel_df = pd.DataFrame(ideal_df)
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Max Temp,Lat,Lng,Wind Speed
0,11,Hermanus,0,ZA,1579922195,96,64.00,-34.42,19.23,1.01
1,20,Bonthe,0,SL,1579922367,89,75.78,7.53,-12.51,2.77
2,33,Taltal,0,CL,1579922370,79,69.78,-25.40,-70.48,2.10
3,49,Cidreira,0,BR,1579922374,70,72.99,-30.18,-50.21,2.13
4,81,Rocha,0,UY,1579922381,49,70.83,-34.48,-54.33,7.90
5,83,Port Alfred,0,ZA,1579922351,94,64.00,-33.59,26.89,2.30
6,98,Pozo Colorado,0,PY,1579922385,50,75.99,-23.49,-58.80,4.72
7,128,Upata,0,VE,1579922392,79,70.50,8.01,-62.40,3.91
8,190,Nicoya,0,CR,1579922407,73,77.00,10.15,-85.45,3.36
9,229,Port Lincoln,0,AU,1579922180,48,75.43,-34.73,135.87,3.85


In [155]:
# search googleplaces for hotels within 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
# list for hotel names
hotels = []
#https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY

# for loop to search through city coords
for cities in hotel_df['City']:

    target_coordinates = (hotel_df['Lat']),(hotel_df['Lng'])
    #print(target_coordinates)
    target_radius = 5000
    target_type = 'lodging'   
    
    params = {"location": target_coordinates,
              "radius": target_radius,
              "type": target_type,
               "key": g_key
             }
    #query_url = base_url + params
# try/except to check if a city turns up    
    try:
        response = requests.get(base_url, params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
       # print("No Lodging Found")
        pass
        #hotels.append(np.nan)
print(target_coordinates)
#append dataframe with hotel names
hotel_df['Hotel Name'] = hotels
hotel_df

(0    -34.42
1      7.53
2    -25.40
3    -30.18
4    -34.48
5    -33.59
6    -23.49
7      8.01
8     10.15
9    -34.73
10     6.30
11    24.14
12   -23.72
13   -32.03
14   -21.48
15     9.27
16   -25.35
Name: Lat, dtype: float64, 0      19.23
1     -12.51
2     -70.48
3     -50.21
4     -54.33
5      26.89
6     -58.80
7     -62.40
8     -85.45
9     135.87
10    -10.80
11     67.45
12     44.38
13    -52.10
14    -56.14
15     -3.00
16    -54.24
Name: Lng, dtype: float64)


ValueError: Length of values does not match length of index

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map